# TimescaleVector

>[TimescaleVector](https://github.com/timescale/python-vector) is a vector similarity search for `PostgreSQL` and `pgvector`. It is availabe as a service on [Timescale Cloud](https://cloud.timescale.com).

TimescaleVector is the vector store of choice for production workloads. It is built on-top of a rock-solid PostgreSQL foundation which provides enterprise-grade feature liked streaming backups and replication, high-availability and row-level security. Timescale adds a brand new indexing method for vector search which provides top-of-the-line performance as well as partitioning methods to allow you to scale your workloads as you grow.

Timescale Cloud offers a 90-day free trial for TimescaleVector users. The platform provides a best-in-class database experience with high-uptime and enterprise features like SOC2 compliance.

This notebook shows how to use the Postgres vector database (`TimescaleVector`).

See the [installation instruction](https://github.com/timescale/python-vector).

In [1]:
# Pip install necessary package
!pip install timescale-vector
!pip install openai
!pip install tiktoken

We want to use `OpenAIEmbeddings` so we have to get the OpenAI API Key.

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [3]:
## Loading Environment Variables
from typing import List, Tuple
from dotenv import load_dotenv

load_dotenv()

False

In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.timescalevector import TimescaleVector
from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document

In [5]:
loader = TextLoader("../../../extras/modules/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [9]:
# Timescale Vector needs the service url to your cloud database. You can see this as soon as you create the 
# service in the cloud UI or in your credentials.sql file
SERVICE_URL = "postgres://tsdbadmin:<password>@<id>.tsdb.cloud.timescale.com:<port>/tsdb?sslmode=require"
SERVICE_URL = "postgres://cevian@localhost:28815/timescaledb_vector"

# # You can get it from an enviornment variables. We suggest using a .env file.
# import os

# SERVICE_URL = os.environ.get("TIMESCALE_SERVICE_URL", "")

## Similarity Search with Euclidean Distance (Default)

In [11]:
# The TimescaleVector Module will try to create a table with the name of the collection.
# So, make sure that the collection name is unique and the user has the permission to create a table.

COLLECTION_NAME = "state_of_the_union_test"

db = TimescaleVector.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=COLLECTION_NAME,
    service_url=SERVICE_URL,
)

In [12]:
query = "What did the president say about Ketanji Brown Jackson"
docs_with_score = db.similarity_search_with_score(query)

In [13]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.18443869404177493
Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
--------------------------------------------------------------------------------
----------------------

## Working with vectorstore

Above, we created a vectorstore from scratch. However, often times we want to work with an existing vectorstore.
In order to do that, we can initialize it directly.

In [15]:
store = TimescaleVector(
    collection_name=COLLECTION_NAME,
    service_url=SERVICE_URL,
    embedding_function=embeddings,
)

### Add documents
We can add documents to the existing vectorstore.

In [16]:
store.add_documents([Document(page_content="foo")])

['42d226f0-4053-11ee-91b6-6ee10b77fd07']

In [17]:
docs_with_score = db.similarity_search_with_score("foo")

In [18]:
docs_with_score[0]

(Document(page_content='foo', metadata={}), 0.0)

In [19]:
docs_with_score[1]

(Document(page_content='A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. \n\nAnd if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. \n\nWe can do both. At our border, we’ve installed new technology like cutting-edge scanners to better detect drug smuggling.  \n\nWe’ve set up joint patrols with Mexico and Guatemala to catch more human traffickers.  \n\nWe’re putting in place dedicated immigration judges so families fleeing persecution and violence can have their cases heard faster. \n\nWe’re securing commitments and supporting partners in South and Central America to host more refugees and secure their own borders.', metadata={'source': '../../../extras/modules/sta

### Overriding a vectorstore

If you have an existing collection, you override it by doing `from_documents` and setting `pre_delete_collection` = True

In [21]:
db = TimescaleVector.from_documents(
    documents=docs,
    embedding=embeddings,
    collection_name=COLLECTION_NAME,
    service_url=SERVICE_URL,
    pre_delete_collection=True,
)

In [22]:
docs_with_score = db.similarity_search_with_score("foo")

In [23]:
docs_with_score[0]

(Document(page_content='A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. \n\nAnd if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. \n\nWe can do both. At our border, we’ve installed new technology like cutting-edge scanners to better detect drug smuggling.  \n\nWe’ve set up joint patrols with Mexico and Guatemala to catch more human traffickers.  \n\nWe’re putting in place dedicated immigration judges so families fleeing persecution and violence can have their cases heard faster. \n\nWe’re securing commitments and supporting partners in South and Central America to host more refugees and secure their own borders.', metadata={'source': '../../../extras/modules/sta

### Using a VectorStore as a Retriever

In [24]:
retriever = store.as_retriever()

In [25]:
print(retriever)

tags=['TimescaleVector', 'OpenAIEmbeddings'] metadata=None vectorstore=<langchain.vectorstores.timescalevector.TimescaleVector object at 0x12b488850> search_type='similarity' search_kwargs={}


## Advanced Usage



### Speeding up queries by creating an index

You can speed up similarity queries by creating an index on the embedding column. You should only do this once you have ingested a large part of your data.

In [26]:
store.create_ivfflat_index()